In [1]:
cd ..

/HDD/ECG_physionet


In [ ]:
#### insert the name of the last run
!tensorboard --logdir=runs/May11_01-21-42_dmitrii

In [2]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import gc
import ast
import json
from data_generator import PreProcessing

# Train

In [3]:
DATA_LIST = [
    '0_fold_0.19386279658302819.txt'
]

DEBUG_PATH = './data/CV_debug/'
DATA_PATH = './data/formatted/'

preprocessing = PreProcessing()

In [4]:
def load_data():
    
    for index,i in enumerate(DATA_LIST):
        if index == 0:
            with open(DEBUG_PATH+i, 'r') as f:
                s = f.read()
                data = ast.literal_eval(s)
        else:
            with open(DEBUG_PATH+i, 'r') as f:
                s = f.read()
                temp = ast.literal_eval(s)
            data.update(temp) 
        
    return data

data = load_data()

In [5]:
#calculate the score for each record
scores = np.array([])
records = np.array([]).astype(np.str)

for record in data.keys():
    scores = np.append(scores,np.sum(np.abs(np.array(data[record]['labels'],dtype=np.float) - np.array(data[record]['preds'],dtype=np.float))))
    records = np.append(records,record)  
                       
#get 50 top worse examples
score_order = np.argsort(scores)
#TODO
records = records[score_order[::1]][:10]

In [6]:
%matplotlib qt

In [7]:
def plot_record(record,data):
    
    ecg = np.load(DATA_PATH+record+'.npy')
    ecg = preprocessing.run(ecg)
    meta = json.load(open(DATA_PATH+ f'{record}.json'))
    
    heatmap = np.array(data[record]['heatmap'],dtype=np.float)
    
    #plot the data
    fig,ax = plt.subplots(figsize=(20,20))
    fig.suptitle(record+', labels: '+str(meta['labels']))
    for i in range(12):
        ax.plot(ecg[:,i]-2.5*i)
    ax.plot(heatmap-2.5*12)
    plt.show()
    
    return 0

plot_record(record=records[0],data=data)

0